# Overview

- this is a baseline model to determine immediate performance
- will use gradient boost modelling using package XGboost
- no data cleaning feature engineering to be used

In [1]:
# General Imports

import numpy as np
import pandas as pd
import pickle

import os, gc, sys, warnings, random, psycopg2

In [2]:
# Functions

def seed_everything(seed=0):
    random.seed(seed)
    np.random.seed(seed)

In [3]:
SEED = 42
LOCAL_TEST = False
seed_everything(SEED)
TARGET = 'meter_reading'

In [4]:
# open pickle files

train_df = pd.read_pickle('train_df.pkl')
test_df = pd.read_pickle('test_df.pkl')
submission = pd.read_pickle('submission.pkl')

In [5]:
#data check#data check
print("Main:",list(train_df),train_df.info())
print("#" * 20)
print("Main:",list(test_df),test_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20216100 entries, 0 to 20216099
Data columns (total 23 columns):
 #   Column              Dtype         
---  ------              -----         
 0   building_id         int16         
 1   site_id             int8          
 2   meter               int8          
 3   meter_reading       float64       
 4   timestamp           datetime64[ns]
 5   primary_use         int8          
 6   square_feet         int32         
 7   year_built          int8          
 8   floor_count         int8          
 9   air_temperature     float64       
 10  cloud_coverage      float16       
 11  dew_temperature     float64       
 12  precip_depth_1_hr   float16       
 13  sea_level_pressure  float64       
 14  wind_direction      float16       
 15  wind_speed          float64       
 16  DT_month            int8          
 17  DT_week_y           int8          
 18  DT_day_y            int8          
 19  DT_hour             int8          
 20  

In [6]:
# Check memory usage

In [9]:
#prepare training data
from xgboost import XGBRegressor

#choose columns to remove
remove_columns = [TARGET,'timestamp']
#get feature columns using list comprehension to remove columns - list(train_df)
feature_columns = [col for col in list(train_df) if col not in remove_columns]

#label is target

x_data = train_df[feature_columns]
y_data = train_df[TARGET]

In [10]:
# Model Parameters

model = XGBRegressor(eta=0.1,verbosity=1)
model.fit(x_data,y_data)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, eta=0.1, gamma=0,
             gpu_id=-1, importance_type='gain', interaction_constraints='',
             learning_rate=0.100000001, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=16, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='approx', validate_parameters=1, verbosity=1)

In [11]:
y_pred=model.predict(x_data)

In [13]:
y_pred

array([1074.6262  ,   85.194305,   70.83401 , ...,   29.856537,
         -8.04701 ,  -25.678986], dtype=float32)

In [15]:
results = pd.DataFrame(data = {'actual':y_data,'predictions':y_pred})

In [16]:
results

,actual,predictions
0,11.1132,1074.626221
1,0.0000,85.194305
2,0.0000,70.834007
3,0.0000,71.751976
4,0.0000,113.240677
...,...,...
20216095,76.0000,93.909164
20216096,182.9460,35.513374
20216097,88.4500,29.856537
20216098,16.0000,-8.047010
